In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 2.9 MB/s eta 0:00:00


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.8 MB/s eta 0:00:00


In [3]:
from huggingface_hub import notebook_login
import pandas as pd
import json
from datasets import load_dataset, DatasetDict, load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
from tqdm.auto import tqdm
import numpy as np
import collections
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from datasets import load_metric

In [4]:
notebook_login()

## Load Dataset

## Dataset can be found under the Data folder

In [11]:
train_data = pd.read_json('/content/train.jsonl',lines=True)
validate_data = pd.read_json('/content/validation.jsonl',lines=True)

In [12]:
#filter only phrase tags for Q&A task
train_data['tags']=train_data['tags'].apply(lambda x:x[0])
train_data=train_data[train_data['tags']=='phrase'].reset_index(drop=True)

#filter only phrase tags for Q&A task
validate_data['tags']=validate_data['tags'].apply(lambda x:x[0])
validate_data=validate_data[validate_data['tags']=='phrase'].reset_index(drop=True)

In [17]:
train_data

,uuid,postId,postText,postPlatform,targetParagraphs,targetTitle,targetDescription,targetKeywords,targetMedia,targetUrl,provenance,spoiler,spoilerPositions,tags
0,b1a1f63d-8853-4a11-89e8-6b2952a393ec,411701128456593408,[NASA sets date for full recovery of ozone hole],Twitter,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[http://s.m.huffpost.com/assets/Logo_Huffingto...,http://huff.to/1cH672Z,"{'source': 'anonymized', 'humanSpoiler': '2070...",[2070],"[[[0, 0], [0, 4]]]",phrase
1,008b7b19-0445-4e16-8f9e-075b73f80ca4,380537005123190784,[This is what makes employees happy -- and it'...,Twitter,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[http://i.huffpost.com/gen/1359674/images/o-HA...,http://huff.to/1epfeaw,"{'source': 'anonymized', 'humanSpoiler': 'Inte...",[intellectual stimulation],"[[[1, 186], [1, 210]]]",phrase
2,31b108a3-c828-421a-a4b9-cf651e9ac859,814186311573766144,[The perfect way to cook rice so that it's per...,Twitter,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",[in a rice cooker],"[[[5, 60], [5, 76]]]",phrase
3,46aa8d72-fbd4-4796-a85b-0a0363c21812,841679598526091265,[Analysis: This may be the most brutal number ...,Twitter,[Plenty has been made of the big Congressional...,This may be the most brutal number in the CBO ...,A 750 percent premium increase for the Trump C...,"ahca, health care, aaron blake, the fix, obama...",None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",[750 percent],"[[[2, 109], [2, 120]]]",phrase
4,917e1106-413c-4be5-818b-ad500314feaa,430433533899710464,[#TeenMom2 star @PBandJenelley_1 reveals the s...,Twitter,"[""Teen Mom 2"" star Jenelle Evans took to Twitt...",'Teen Mom 2' Star Jenelle Evans Reveals Sex Of...,"""Teen Mom 2"" star Jenelle Evans took to Twitte...","jenelle evans teen mom,jenelle evans,jenelle e...",[http://s.m.huffpost.com/assets/Logo_Huffingto...,http://huff.to/1fFY7QD,"{'source': 'anonymized', 'humanSpoiler': 'Boy'...",[boy],"[[[0, 103], [0, 106]]]",phrase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1362,edec6cde-bbd3-4f50-ba51-ef98bdf19b63,4m7ole,"[He Dug A Huge Hole In His Backyard, And What ...",reddit,"[When it comes to home projects, sometimes the...","He Dug A Huge Hole In His Backyard, And What H...",None,None,[https://n-cdn.areyouahuman.com/noscript/a3c26...,https://thedesigninspiration.com/news/awesome/...,"{'source': 'anonymized', 'humanSpoiler': 'Buil...",[own underground bunker!],"[[[20, 70], [20, 93]]]",phrase
1363,84277c0f-4ef1-444e-8e04-55c84af593e5,4lnjcw,[Best Buy Has An Insane Xbox One Deal For A Li...,reddit,[Best Buy Has An Insane Xbox One Deal For A Li...,Best Buy Has An Insane Xbox One Deal For A Lim...,For one week only Best Buy has a special $50 o...,"entertainment news, movie news, movie reviews,...",[http://b.scorecardresearch.com/p?c1=2&c2=2122...,http://webcache.googleusercontent.com/search?q...,"{'source': 'anonymized', 'humanSpoiler': '$50....",[$50 off],"[[[8, 101], [8, 108]]]",phrase
1364,5b61d712-8b03-4ee6-ba04-b39ce2b206f7,387429931400953856,[Student forced to carry papers to prove she c...,Twitter,[She's not a tourist visiting a foreign countr...,"Melona Clark, Hampton University Student, Carr...",She's not a tourist visiting a foreign country...,"student headscarf university,student hijab col...",[http://i.huffpost.com/gen/1392630/images/n-ME...,http://huff.to/1bRnSLB,"{'source': 'anonymized', 'humanSpoiler': 'At H...",[student at Hampton University in Virginia],"[[[1, 9], [1, 50]]]",phrase
1365,9d0

In [14]:
# data processing to get context and spoiler start position
def create_df(dataset):

    col1=[]
    col2=[]

    for idx in range(len(dataset)):
        concatlen=0
        line_occuring=dataset['spoilerPositions'][idx][0][0][0]

        for i in range(line_occuring):
            concatlen+=len(dataset['targetParagraphs'][idx][i])


        starting_position=concatlen+dataset['spoilerPositions'][idx][0][0][1]

        col1.append({'answer_start': [starting_position], 'text': dataset['spoiler'][idx]})


        l=''
        for line in dataset['targetParagraphs'][idx]:
            l+=line
        
        col2.append(l)

    df = pd.DataFrame(list(zip(col1, col2)),
    columns =['answers', 'context'])
    df['question']=dataset['postText'].apply(lambda x: x[0])
    df['id']=dataset['postId']

    json_data=[]
    for index, row in df.iterrows():
        json_data.append({
            'question': row['question'],
            'context': row['context'],
            'answers': row['answers'],
            'id': row['id']
        })

    return json_data

In [15]:
train_json=create_df(train_data)
validate_json=create_df(validate_data)

with open('train_json.json', 'w') as f:
    json.dump(train_json, f)

with open('validate_json.json', 'w') as f:
    json.dump(validate_json, f)

In [20]:
# Load the JSON file as a Dataset object
train_dataset = load_dataset('json', data_files='train_json.json')


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-d28ef1d7cd4751e1/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
validate_dataset = load_dataset('json', data_files='validate_json.json')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-0264ff07317e6fca/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
# Create a DatasetDict object containing the Dataset
train_dict = DatasetDict({'train': train_dataset['train']})
valid_dict = DatasetDict({'valid': validate_dataset['train']})

## Data Pre-Processing

In [23]:
model = "deepset/deberta-v3-large-squad2"
tokenizer = AutoTokenizer.from_pretrained(model)

In [24]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [26]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [27]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [28]:
tokenized_train = train_dict.map(prepare_train_features, batched=True, remove_columns=train_dict["train"].column_names)
tokenized_valid = valid_dict.map(prepare_validation_features, batched=True, remove_columns=valid_dict["valid"].column_names)

Map:   0%|          | 0/1367 [00:00<?, ? examples/s]

Map:   0%|          | 0/335 [00:00<?, ? examples/s]

## Training

In [29]:
model = AutoModelForQuestionAnswering.from_pretrained(model)

In [30]:
batch_size=2

In [31]:
model_name = "deepset/deberta-v3-large-squad2"
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=True,
)

In [32]:

data_collator = default_data_collator

In [33]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train["train"],
    eval_dataset=tokenized_valid["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/Lahen/deberta-v3-large-squad2-finetuned-squad into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.00k/1.62G [00:00<?, ?B/s]

Download file runs/Apr25_21-48-06_0d78a5323319/1682459297.1007872/events.out.tfevents.1682459297.0d78a5323319.…

Download file training_args.bin: 100%|##########| 3.56k/3.56k [00:00<?, ?B/s]

Download file runs/Apr25_21-48-06_0d78a5323319/events.out.tfevents.1682459297.0d78a5323319.1100.0: 100%|######…

Clean file runs/Apr25_21-48-06_0d78a5323319/1682459297.1007872/events.out.tfevents.1682459297.0d78a5323319.110…

Clean file training_args.bin:  28%|##8       | 1.00k/3.56k [00:00<?, ?B/s]

Clean file runs/Apr25_21-48-06_0d78a5323319/events.out.tfevents.1682459297.0d78a5323319.1100.0:  21%|##       …

Clean file pytorch_model.bin:   0%|          | 1.00k/1.62G [00:00<?, ?B/s]

In [34]:
trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.127000,No log


TrainOutput(global_step=1536, training_loss=1.398257628083229, metrics={'train_runtime': 1829.1124, 'train_samples_per_second': 1.68, 'train_steps_per_second': 0.84, 'total_flos': 2139755405377536.0, 'train_loss': 1.398257628083229, 'epoch': 1.0})

## Model

In [35]:
trainer.save_model("deberta-trained")

Upload file pytorch_model.bin:   0%|          | 1.00/1.62G [00:00<?, ?B/s]

Upload file runs/Apr25_23-46-44_ba6854deac76/events.out.tfevents.1682466762.ba6854deac76.6028.0:   0%|        …

To https://huggingface.co/Lahen/deberta-v3-large-squad2-finetuned-squad
   24981f5..0972d4a  main -> main

   24981f5..0972d4a  main -> main

To https://huggingface.co/Lahen/deberta-v3-large-squad2-finetuned-squad
   0972d4a..58fa556  main -> main

   0972d4a..58fa556  main -> main



In [44]:
# !zip -r deberta-trained.zip '/content/deberta-trained'


  adding: content/deberta-trained/ (stored 0%)
  adding: content/deberta-trained/tokenizer_config.json (deflated 46%)
  adding: content/deberta-trained/training_args.bin (deflated 48%)
  adding: content/deberta-trained/config.json (deflated 55%)
  adding: content/deberta-trained/special_tokens_map.json (deflated 54%)
  adding: content/deberta-trained/pytorch_model.bin (deflated 9%)
  adding: content/deberta-trained/tokenizer.json (deflated 77%)


## Prediction

In [36]:


def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not True:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [37]:
raw_predictions = trainer.predict(tokenized_valid['valid'])

In [38]:
final_predictions = postprocess_qa_predictions(valid_dict['valid'], tokenized_valid['valid'], raw_predictions.predictions)

Post-processing 335 example predictions split into 735 features.


  0%|          | 0/335 [00:00<?, ?it/s]

In [39]:
#to see output. comment if not needed.

final_predictions

OrderedDict([('857559227384160256', ''),
             ('388301310635671553', 'Sprite'),
             ('553676574290354177', 'Smoky Paprika-Baked Garbanzo Beans'),
             ('4kt69b', ' Anthony Bourdain'),
             ('4wagdb', ''),
             ('1100188040002123_1123063127714614', ''),
             ('473452702002065409', ''),
             ('170016066732231_174009032999601', ''),
             ('474005518713769985', ' Mike Rounds'),
             ('4kq6db', ' Jalapeno Popper Quesarito'),
             ('438689079539286016', ''),
             ('403757856597372928', ' Rag & Bone'),
             ('398323375371608064', ' pixie cut'),
             ('4wennf', ' southern flying squirrel'),
             ('384053590207119361', ' antique cabinet'),
             ('590544984098635776', ' August 6th'),
             ('555426985967894529', ''),
             ('452637109074272256', ''),
             ('54zbhl', 'The Arizona Republic'),
             ('379437666896728064', ''),
             ('110018804

## Evaluation

In [40]:
predictions = [ v for k, v in final_predictions.items()]
references = [ex["answers"]["text"] for ex in valid_dict['valid']]


In [41]:

# remove empty string elements from predictions and references
indices_to_remove = [i for i, p in enumerate(predictions) if p == ""]
for i in reversed(indices_to_remove):
    del predictions[i]
    del references[i]

print(predictions)  
print(references) 

['Sprite', 'Smoky Paprika-Baked Garbanzo Beans', ' Anthony Bourdain', ' Mike Rounds', ' Jalapeno Popper Quesarito', ' Rag & Bone', ' pixie cut', ' southern flying squirrel', ' antique cabinet', ' August 6th', 'The Arizona Republic', ' Tim Masthay', ' squash', ' $25K', ' Crazy Frog', ' Dieffenbachia', ' baby food', ' Kirk Douglass', ' 1,4-dioxane', ' Wyoming', '2.1 coffee drinks per day', ' giant piece of cardboard', ' Switzerland', ' make-shift tap shoes', ' two foundations', ' Boxed', ' $700 million', ' Gilbert Arenas', ' Alex Castellanos', ' "Gravity', ' Wisconsin', ' Isla Fisher', ' never made a dollar', ' Atlanta', ' Israelis', ' Chiwetel Ejiofor', ' Lviv', ' joins in', ' June 12 at 5 PM ET', ' R. Kelly', ' $1,000 per haircut', ' Wiz Khalifa', ' Fab, right?', ' Let Hillary live!', 'VLC', ' 19 warm sweaters', ' Black Women', "I think she's fabulous", ' dying his hair sky blue', ' underarm deodorant', 'Arthur Brooks', 'Antenna Bands', ' his own clothes', " China's Sichuan Province", 

### BLEU Score

In [18]:
bleuscore = sentence_bleu(references, predictions, weights=[.2,0,0,0])
print(f'BLEU Score:',bleuscore)

BLEU Score: 0.6543893899412373


### METEOR Score

In [19]:
meteor = load_metric("meteor")
score = meteor.compute(predictions=predictions, references=references)
alpha=.6
weighted_score = (1 - alpha) * score['meteor'] + alpha * len(predictions) / len(references)
weighted_score

<ipython-input-19-11a95073a650>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  meteor = load_metric("meteor")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


0.6604630017067953